In [1]:
original_price = [1, 5, 8, 9, 10, 17, 17, 20, 24, 30]

In [4]:
price = {}
for i,p in enumerate(original_price):
    price[i+1] = p

In [5]:
price[2]

5

In [6]:
price[11]

KeyError: 11

In [7]:
from collections import defaultdict

In [8]:
some_default_dict = defaultdict(lambda : 'None')

In [9]:
some_default_dict[12]

'None'

In [12]:
price = defaultdict(int)
for i, p in enumerate(original_price):
    price[i+1] = p

In [13]:
price[11]

0

## get the max splitting by enumerate

In [26]:
def r(n):
    return max([price[n]]+[r(i)+r(n-i) for i in range(1,n)])

In [27]:
r(4)

10

In [30]:
def r(n):
    return max(price[n], max(r(i)+r(n-i) for i in range(1,n)))

In [31]:
r(4)

ValueError: max() arg is an empty sequence

## r(n) 被调用的次数

In [34]:
def example(f,arg):
    return f(arg)
def add_ten(num):
    return num+10
def mul_ten(num):
    return num*10

In [33]:
example(add_ten,100)

110

In [39]:
operations = [add_ten, mul_ten]
for f in operations:
    print(example(f,100))

110
1000


In [46]:
called_time = defaultdict(int)
def get_call_times(f):
    result = f()
    print('function:{} called once'.format(f.__name__))
    called_time[f.__name__] += 1
    return result

In [47]:
def some_function_1():print('I am function1')

In [48]:
get_call_times(some_function_1)

I am function1
function:some_function_1 called once


In [49]:
called_time

defaultdict(int, {'some_function_1': 1})

In [50]:
get_call_times(some_function_1)

I am function1
function:some_function_1 called once


In [51]:
called_time

defaultdict(int, {'some_function_1': 2})

In [52]:
call_time_with_arg = defaultdict(int)

In [54]:
def r(n):
    fname = r.__name__
    call_time_with_arg[(fname,n)]+=1
    return max([price[n]]+[r(i)+r(n-i) for i in range(1,n)])

In [55]:
r(15)

43

In [60]:
from collections import Counter

In [62]:
Counter(call_time_with_arg).most_common()

[(('r', 1), 3188646),
 (('r', 2), 1062882),
 (('r', 3), 354294),
 (('r', 4), 118098),
 (('r', 5), 39366),
 (('r', 6), 13122),
 (('r', 7), 4374),
 (('r', 8), 1458),
 (('r', 9), 486),
 (('r', 10), 162),
 (('r', 11), 54),
 (('r', 12), 18),
 (('r', 13), 6),
 (('r', 14), 2),
 (('r', 15), 1)]

In [70]:
call_time_with_arg = defaultdict(int)
def get_call_time(f):
    def wrap(n):
        print('I can count')
        result = f(n)
        call_time_with_arg[(f.__name__,n)]+=1
        return result
    return wrap
def add_ten(num):
    return num+10

In [71]:
get_call_time(add_ten)

<function __main__.get_call_time.<locals>.wrap(n)>

In [72]:
get_call_time(add_ten)(10)

I can count


20

In [67]:
call_time_with_arg

defaultdict(int, {('add_ten', 10): 1})

In [68]:
get_call_time(add_ten)(9)

19

In [69]:
call_time_with_arg

defaultdict(int, {('add_ten', 10): 1, ('add_ten', 9): 1})

In [73]:
@get_call_time
def add_ten(num):
    return num+10

In [74]:
add_ten(9)

I can count


19

In [79]:
call_time_with_arg = defaultdict(int)
def get_call_time(f):
    def wrap(n):
        '''
        haha i am wrap
        '''
        #print('I can count')
        result = f(n)
        call_time_with_arg[(f.__name__,n)]+=1
        return result
    return wrap

In [80]:
@get_call_time
def r(n):
    """
    n is the length of iron
    it returns the max revenue
    """
    return max([price[n]]+[r(i)+r(n-i) for i in range(1,n)])

In [81]:
r

<function __main__.get_call_time.<locals>.wrap(n)>

### 使用@get_call_time失去了r函数命名的意义

In [82]:
help(r) 

Help on function wrap in module __main__:

wrap(n)
    haha i am wrap



In [77]:
r(10)

30

In [78]:
call_time_with_arg

defaultdict(int,
            {('r', 1): 13122,
             ('r', 2): 4374,
             ('r', 3): 1458,
             ('r', 4): 486,
             ('r', 5): 162,
             ('r', 6): 54,
             ('r', 7): 18,
             ('r', 8): 6,
             ('r', 9): 2,
             ('r', 10): 1})

### 使用@wraps装饰器

In [85]:
from functools import wraps

In [86]:
def get_call_time(f):
    @wraps(f)
    def wrap(n):
        '''
        haha i am wrap
        '''
        #print('I can count')
        result = f(n)
        call_time_with_arg[(f.__name__,n)]+=1
        return result
    return wrap

In [87]:
@get_call_time
def r(n):
    """
    n is the length of iron
    it returns the max revenue
    """
    return max([price[n]]+[r(i)+r(n-i) for i in range(1,n)])

In [88]:
help(r) 

Help on function r in module __main__:

r(n)
    n is the length of iron
    it returns the max revenue



### 使用装饰器加入cache

In [144]:
#cache = {}
def memo(f):
    cache = {}
    @wraps(f)
    def _wrap(n):
        if n in cache: return cache[n]
        else: 
            cache[n] = f(n)
            return f(n)
    return _wrap
"""
def memo(func):
    cache = {}
    @wraps(func)
    def _wrap(n): ## ? *args, **kwargs
        if n in cache: result = cache[n]
        else:
            result = func(n)
            cache[n] = result
        return result
    return _wrap
"""

'\ndef memo(func):\n    cache = {}\n    @wraps(func)\n    def _wrap(n): ## ? *args, **kwargs\n        if n in cache: result = cache[n]\n        else:\n            result = func(n)\n            cache[n] = result\n        return result\n    return _wrap\n'

In [116]:
@memo
def r(n):
    """
    n is the length of iron
    it returns the max revenue
    """
    return max([price[n]]+[r(i)+r(n-i) for i in range(1,n)])

In [117]:
r(15)

43

In [122]:
call_time_with_arg = defaultdict(int)
@get_call_time
@memo
def r(n):
    """
    n is the length of iron
    it returns the max revenue
    """
    return max([price[n]]+[r(i)+r(n-i) for i in range(1,n)])

In [123]:
r(19)

55

In [124]:
call_time_with_arg

defaultdict(int,
            {('r', 1): 72,
             ('r', 2): 68,
             ('r', 3): 64,
             ('r', 4): 60,
             ('r', 5): 56,
             ('r', 6): 52,
             ('r', 7): 48,
             ('r', 8): 44,
             ('r', 9): 40,
             ('r', 10): 36,
             ('r', 11): 32,
             ('r', 12): 28,
             ('r', 13): 24,
             ('r', 14): 20,
             ('r', 15): 16,
             ('r', 16): 12,
             ('r', 17): 8,
             ('r', 18): 4,
             ('r', 19): 1})

### 找出如何切分的

In [161]:
solution = {}

@memo
def r(n):
    """
    n is the length of iron
    it returns the max revenue
    """
    max_price, max_split = max([(price[n],0)]+[(r(i)+r(n-i),i) for i in range(1,n)],key = lambda x: x[0])
    solution[n] = (max_split,n-max_split) 
    #print(max_split,n-max_split)
    return max_price

In [162]:
r(10)

30

In [169]:
solution

{1: (0, 1),
 2: (0, 2),
 3: (0, 3),
 4: (2, 2),
 5: (2, 3),
 6: (0, 6),
 7: (1, 6),
 8: (2, 6),
 9: (3, 6),
 10: (0, 10)}

In [164]:
cache

{}

In [202]:

def parse_solution(n,revenue_solution):
    left_split, right_split = revenue_solution[n]
    if  left_split == 0:
        return [right_split]
    return parse_solution(left_split,revenue_solution)+parse_solution(right_split,revenue_solution)


In [205]:
parse_solution(9, solution)

[3, 6]